In [50]:
import math
import pandas

In [79]:
reranked = [0, 1, 4 ,2]
max_rank = 4

In [80]:
def d_g(rank_list):
    return [(pow(2, rel) - 1)/(math.log2(i + 2)) for i, rel in enumerate(rank_list)]


DG = d_g(reranked)
ideal = sorted(reranked, reverse=True)
IDG = d_g(ideal)

DCG = sum(DG)
IDCG = sum(IDG)
NDCG = DCG / IDCG 

df_dcg = pandas.DataFrame({
   'rel_i': reranked,
   'DG': DG
})
df_idcg = pandas.DataFrame({
   'rel_i': ideal,
   'IDG': IDG
})

print("===== Reranked ====")
print(df_dcg)
print("===== Ideal ====")
print(df_idcg)
print("===== Result ====")
print('DCG={:.2f}, Ideal DCG={:.2f}, NDCG={:.2f}'.format(DCG, IDCG, NDCG))

===== Reranked ====
   rel_i       DG
0      0  0.00000
1      1  0.63093
2      4  7.50000
3      2  1.29203
===== Ideal ====
   rel_i        IDG
0      4  15.000000
1      2   1.892789
2      1   0.500000
3      0   0.000000
===== Result ====
DCG=9.42, Ideal DCG=17.39, NDCG=0.54


In [81]:
ERR = 0.0
err_list = []
prob_step_down = 1.0
prob_list = []
    
for r, rel in enumerate(reranked):
    r += 1
    R = (pow(2, rel) - 1) / pow(2, max_rank)
    err_step = prob_step_down * R / r
    ERR += err_step
    err_list.append(err_step)

    prob_step_down *= (1 - R)
    prob_list.append(prob_step_down)
    
df_ERR = pandas.DataFrame({
   'probability': prob_list,
   'ERR': err_list
})
print(reranked)
print('ERR={:.2f}'.format(ERR))
print("===== Iterations ====")
print(df_ERR)

[0, 1, 4, 2]
ERR=0.33
===== Iterations ====
   probability       ERR
0     1.000000  0.000000
1     0.937500  0.031250
2     0.058594  0.292969
3     0.047607  0.002747


In [63]:
precision = sum(1 for rank in reranked if rank > 0) / len(reranked)
print('Precision={:.2f}'.format(precision))

Precision=1.00
